# **Simply Supported Beam:**

**Boundary Conditions:**   `v[x = 0] = 0; v[x = L] = 0; d2v/dx2[x = 0] = 0; d2v/dx2[x = L] = 0`

Explanation:

*   `v[x = 0] = 0` means the beam cannot deflect at the left support because it is pinned.
*   `v[x = L] = 0` means the beam cannot deflect at the right support because it is pinned.

*   `d2v/dx2[x = 0] = 0` means the beam is free to rotate and does not experience bending moment at the left support.

*   `d2v/dx2[x = L] = 0` means the beam is free to rotate, and the bending moment at the right support is zero.

---
Point Load
---

In [ ]:
import numpy as np

def newton_raphson_system(N, init_guess='uniform', tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.
        FD1: Finite Difference Method 1
        FD2: Finite Difference Method 2

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Validate inputs
    valid_init_guesses = ['random', 'uniform', 'zeros']

    if init_guess not in valid_init_guesses:
        raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  2000       # Flexural rigidity (N·m^2) - A measure of a beam's resistance to bending.
    P  = -100.0      # Point load at the middle (N) - The force applied at the center of the beam.
    w  =  0.5        # Relaxation factor - Used to control the step size in the Newton-Raphson iteration, often to aid convergence.

    # Initial guess for the deflection v(x)
    if init_guess == 'random':
        # Random initial guess, scaled by the sign of P for appropriate direction
        v = np.random.uniform(0, 0.01 * np.sign(P), N+1)
        v[0] = 0  # Boundary condition: deflection at x=0 is zero
        v[-1] = 0 # Boundary condition: deflection at x=L is zero
    elif init_guess == 'uniform':
        # Uniform initial guess, linearly varying and scaled by the sign of P
        v = 0.0001 * np.sign(P) * x[:]
        v[0] = 0  # Boundary condition: deflection at x=0 is zero
        v[-1] = 0 # Boundary condition: deflection at x=L is zero
    elif init_guess == 'zeros':
        # Initial guess of all zeros
        v = np.zeros(N+1)
        v[0] = 0  # Boundary condition: deflection at x=0 is zero
        v[-1] = 0 # Boundary condition: deflection at x=L is zero
    print()


    # Moment Equation for a simply supported beam with a point load at the center
    M = np.zeros(N+1)
    for i in range(N+1):
        if x[i] <= L/2:
            M[i] = (P/2) * x[i] # Moment for the first half of the beam
        else:
            M[i] = (P/2) * (L - x[i]) # Moment for the second half of the beam

    # Residual Function (Using only FD2 method for the main equation, and specific FDM for boundary conditions)
    def residual(v, M):
        """Compute the residual vector R(v).
           The residual represents how far the current approximation deviates from the actual solution.
        """
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Enforcing boundary condition v(0) = 0
        v_full[-1] = 0          # Enforcing boundary condition v(L) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition: d2v/dx2[x=0] = 0 (zero curvature at x=0 for simply supported beam)
                # Using a forward finite difference approximation for the second derivative.
                R[i] = (2*v_full[i]-5*v_full[i+1]+4*v_full[i+2]-v_full[i+3])/dx**2
            elif i == N:
                # Boundary condition: d2v/dx2[x=L] = 0 (zero curvature at x=L for simply supported beam)
                # Using a backward finite difference approximation for the second derivative.
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])/dx**2
            else:
                # Main governing equation for beam deflection (nonlinear, considering large deflections)
                # This is derived from the moment-curvature relationship: d2v/dx2 = M/EI * (1 + (dv/dx)^2)^(3/2)
                # The term (v_full[i+1]-v_full[i-1])/(2*dx) approximates dv/dx.
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v).
           The Jacobian matrix contains the partial derivatives of the residual components with respect to the deflection values.
           It's crucial for the Newton-Raphson method to linearize the nonlinear system.
        """
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Enforcing boundary condition v(0) = 0
        v_full[-1] = 0          # Enforcing boundary condition v(L) = 0

        for i in range(0, N+1):
            if i == 0:
                # Derivatives of R[0] with respect to v_full[i], v_full[i+1], v_full[i+2], v_full[i+3]
                # These correspond to the finite difference coefficients for the boundary condition.
                J[i, i], J[i, i+1], J[i, i+2], J[i, i+3] =  0, -5/dx**2, 4/dx**2, -1/dx**2
            elif i == N:
                # Derivatives of R[N] with respect to v_full[i], v_full[i-1], v_full[i-2], v_full[i-3]
                # These correspond to the finite difference coefficients for the boundary condition.
                J[i, i], J[i, i-1], J[i, i-2], J[i, i-3] =  0, -5/dx**2, 4/dx**2, -1/dx**2
            else:
                # Derivatives of the main residual equation R[i] with respect to v_full[i-1], v_full[i], v_full[i+1]
                # This involves the derivative of the nonlinear term.
                if i == 1:
                    # Special case for i=1 to avoid accessing v_full[i-2] or v_full[i-3] for the boundary condition at i=0.
                    # This ensures the matrix structure is correct.
                    J[i, i-1] = 0
                else:
                    # Derivative with respect to v_full[i-1]
                    J[i, i-1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i-1]-2*v_full[i+1])
                # Derivative with respect to v_full[i]
                J[i, i]   = -2/dx**2
                # Derivative with respect to v_full[i+1]
                J[i, i+1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Newton-Raphson iteration loop
    for iteration in range(max_iter):
        R = residual(v, M) # Calculate the residual vector for the current deflection estimate
        print(np.linalg.norm(R)) # Print the L2 norm of the residual (a measure of error)
        if np.linalg.norm(R) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n") # Check for convergence based on residual norm
            break

        J = jacobian(v, M) # Calculate the Jacobian matrix for the current deflection estimate
        try:
            # Attempt to solve the linear system J * delta_v = -R for delta_v
            delta_v = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            # If the Jacobian is singular (not invertible), use the pseudo-inverse (least squares solution)
            print(f"\nJacobian is singular for iteration {iteration+1}. Using pseudo-inverse.")
            delta_v = np.linalg.lstsq(J, -R, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist

        v += w*delta_v # Update the deflection estimate using the relaxation factor (w)

        if np.linalg.norm(delta_v) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n") # Check for convergence based on the change in deflection
            break
    else:
        # If the loop completes without breaking, it means convergence was not achieved
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Final solution with boundary conditions enforced
    v_full = np.zeros(N+1)
    v_full[:] = v
    v_full[0] = 0
    v_full[-1] = 0

    return x, v_full, L, P, EI


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 250 # Number of grid spaces for discretization
init_guess = input("Enter initial guess type (random, uniform, zeros): ").strip().lower() # User input for initial guess
x, v, L, P, EI = newton_raphson_system(N, init_guess) # Call the Newton-Raphson solver
print("The Deflection Matrix is: ")
print(v) # Print the calculated deflection values
print()


# Create Pandas DataFrame for easy plotting
df = pd.DataFrame({'x': x, 'Newton-Raphson Solution (Point Load)': v, 'Euler-Bernoulli Solution (Point Load)': np.where(x <= L/2, (P * x / (12 * EI)) * ((3/4) * L**2 - x**2), (P * (L - x) / (12 * EI)) * (-x**2 + 2*L*x - (1/4) * L**2))})

# Create interactive plot using Plotly for better visualization
fig = px.line(df, x='x', y=df.columns[1:], title='Beam Deflection Solutions')
fig.update_layout(
    xaxis_title='Position (x)', # Label for the x-axis
    yaxis_title='Deflection (v)', # Label for the y-axis
    legend_title='Solution Type' # Title for the legend
)
fig.show() # Display the plot

---
Comparison of Point Load v/s Distributed Load
---

In [ ]:
import numpy as np

def newton_raphson_system(N, init_guess='uniform', tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Validate inputs
    valid_init_guesses = ['random', 'uniform', 'zeros']

    if init_guess not in valid_init_guesses:
        raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  2000       # Flexural rigidity (N·m^2) - A measure of a beam's resistance to bending.
    P  = -100.0      # Point load at the middle (N) - A concentrated force applied at a single point.
    W  = -100.0      # Continuous Load (N/m) - A distributed force acting over a length.
    w  =  0.5        # Relaxation factor - Used to control the step size in the iteration, aiding convergence.

    # Initial guess -- Continuous load
    if init_guess == 'random':
        # Random initial guess for continuous load, scaled by the sign of W.
        v_cont = np.random.uniform(0, 0.01 * np.sign(W), N+1)
        v_cont[0] = 0 # Boundary condition: deflection at x=0 is 0.
        v_cont[-1] = 0 # Boundary condition: deflection at x=L is 0.
    elif init_guess == 'uniform':
        # Uniform initial guess for continuous load, scaled by the sign of W and x.
        v_cont = 0.0001 * np.sign(W) * x[:]
        v_cont[0] = 0 # Boundary condition: deflection at x=0 is 0.
        v_cont[-1] = 0 # Boundary condition: deflection at x=L is 0.
    elif init_guess == 'zeros':
        # Zero initial guess for continuous load.
        v_cont = np.zeros(N+1)
        v_cont[0] = 0 # Boundary condition: deflection at x=0 is 0.
        v_cont[-1] = 0 # Boundary condition: deflection at x=L is 0.

    # Initial guess -- Point load
    if init_guess == 'random':
        # Random initial guess for point load, scaled by the sign of P.
        v_point = np.random.uniform(0, 0.01 * np.sign(P), N+1)
        v_point[0] = 0 # Boundary condition: deflection at x=0 is 0.
        v_point[-1] = 0 # Boundary condition: deflection at x=L is 0.
    elif init_guess == 'uniform':
        # Uniform initial guess for point load, scaled by the sign of P and x.
        v_point = 0.0001 * np.sign(P) * x[:]
        v_point[0] = 0 # Boundary condition: deflection at x=0 is 0.
        v_point[-1] = 0 # Boundary condition: deflection at x=L is 0.
    elif init_guess == 'zeros':
        # Zero initial guess for point load.
        v_point = np.zeros(N+1)
        v_point[0] = 0 # Boundary condition: deflection at x=0 is 0.
        v_point[-1] = 0 # Boundary condition: deflection at x=L is 0.

    # Moment Equation
    M_cont = np.zeros(N+1)
    M_point = np.zeros(N+1)
    for i in range(N+1):
        # Continuous Load: Bending moment calculation for a simply supported beam with a continuous load.
        M_cont[i] = 1/2 * W * x[i] * (L - x[i])

        # Point Load: Bending moment calculation for a simply supported beam with a point load at mid-span.
        if x[i] <= L/2:
            M_point[i] = (P/2) * x[i]
        else:
            M_point[i] = (P/2) * (L - x[i])

    # Residual Function
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0 (Deflection at the start is zero)
        v_full[-1] = 0          # Boundary condition v(L) = 0 (Deflection at the end is zero)

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition for the second derivative of deflection at x=0 is 0 (zero bending moment at the start).
                # This uses a forward finite difference approximation for d^2v/dx^2.
                R[i] = (2*v_full[i]-5*v_full[i+1]+4*v_full[i+2]-v_full[i+3])/dx**2
            elif i == N:
                # Boundary condition for the second derivative of deflection at x=L is 0 (zero bending moment at the end).
                # This uses a backward finite difference approximation for d^2v/dx^2.
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])/dx**2
            else:
                # The main residual equation based on the nonlinear beam bending theory.
                # It relates the curvature (d^2v/dx^2) to the bending moment (M/EI)
                # with a nonlinear term accounting for large deflections.
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0
        v_full[-1] = 0          # Boundary condition v(L) = 0

        for i in range(0, N+1):
            if i == 0:
                # Jacobian entries for the d^2v/dx^2 boundary condition at x=0.
                J[i, i], J[i, i+1], J[i, i+2], J[i, i+3] =  0, -5/dx**2, 4/dx**2, -1/dx**2
            elif i == N:
                # Jacobian entries for the d^2v/dx^2 boundary condition at x=L.
                J[i, i], J[i, i-1], J[i, i-2], J[i, i-3] =  0, -5/dx**2, 4/dx**2, -1/dx**2
            else:
                # Jacobian entries for the main residual equation. These are partial derivatives
                # of the residual with respect to v_i-1, v_i, and v_i+1.
                if i == 1: # Special handling for i=1 to avoid out-of-bounds for J[i, i-1] in this specific Jacobian structure.
                    J[i, i-1] = 0
                else:
                    # Derivative with respect to v_i-1
                    J[i, i-1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i-1]-2*v_full[i+1])
                # Derivative with respect to v_i
                J[i, i]   = -2/dx**2
                # Derivative with respect to v_i+1
                J[i, i+1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Newton-Raphson iteration -- Continuous load
    print("Starting Newton-Raphson for Continuous Load...")
    for iteration in range(max_iter):
        R_cont = residual(v_cont, M_cont) # Compute the residual vector.
        print(f"Continuous Load - Iteration {iteration+1}: Residual norm = {np.linalg.norm(R_cont):.8e}")
        if np.linalg.norm(R_cont) < tol: # Check for convergence based on residual norm.
            print(f"\nConverged in {iteration+1} iterations for Continuous Load.\n")
            break

        J_cont = jacobian(v_cont, M_cont) # Compute the Jacobian matrix.
        try:
            # Attempt to solve the linear system J * delta_v = -R.
            delta_v_cont = np.linalg.solve(J_cont, -R_cont)
        except np.linalg.LinAlgError:
            # If the Jacobian is singular, use the pseudo-inverse (least squares solution).
            print(f"\nJacobian is singular for iteration {iteration+1} (Continuous Load). Using pseudo-inverse.")
            delta_v_cont = np.linalg.lstsq(J_cont, -R_cont, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        v_cont += w*delta_v_cont # Update the solution with relaxation factor.

        if np.linalg.norm(delta_v_cont) < tol: # Check for convergence based on the change in solution.
            print(f"\nConverged in {iteration+1} iterations for Continuous Load.\n")
            break
    else:
        # Raise an error if max_iter is reached without convergence.
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1} for Continuous Load.")

    # Newton-Raphson iteration -- Point load
    print("Starting Newton-Raphson for Point Load...")
    for iteration in range(max_iter):
        R_point = residual(v_point, M_point) # Compute the residual vector.
        print(f"Point Load - Iteration {iteration+1}: Residual norm = {np.linalg.norm(R_point):.8e}")
        if np.linalg.norm(R_point) < tol: # Check for convergence based on residual norm.
            print(f"\nConverged in {iteration+1} iterations for Point Load.\n")
            break

        J_point = jacobian(v_point, M_point) # Compute the Jacobian matrix.
        try:
            # Attempt to solve the linear system J * delta_v = -R.
            delta_v_point = np.linalg.solve(J_point, -R_point)
        except np.linalg.LinAlgError:
            # If the Jacobian is singular, use the pseudo-inverse (least squares solution).
            print(f"\nJacobian is singular for iteration {iteration+1} (Point Load). Using pseudo-inverse.")
            delta_v_point = np.linalg.lstsq(J_point, -R_point, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        v_point += w*delta_v_point # Update the solution with relaxation factor.

        if np.linalg.norm(delta_v_point) < tol: # Check for convergence based on the change in solution.
            print(f"\nConverged in {iteration+1} iterations for Point Load.\n")
            break
    else:
        # Raise an error if max_iter is reached without convergence.
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1} for Point Load.")

    # Full solution including boundaries (ensure boundary conditions are strictly zero)
    v_full_cont = np.zeros(N+1)
    v_full_cont[:] = v_cont
    v_full_cont[0] = 0
    v_full_cont[-1] = 0

    v_full_point = np.zeros(N+1)
    v_full_point[:] = v_point
    v_full_point[0] = 0
    v_full_point[-1] = 0

    return x, v_full_point, v_full_cont, L, P, W, EI


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 250 # Number of grid spaces for discretization.
init_guess = input("Enter initial guess type (random, uniform, zeros): ").strip().lower() # User input for initial guess.
# Call the Newton-Raphson solver function.
x, v_point, v_cont, L, P, W, EI = newton_raphson_system(N, init_guess)
# print(v) # Debug print for the solution vector (commented out).
print()

# Create Pandas DataFrame to store results for easy plotting.
df = pd.DataFrame({'x': x,
                   'Newton-Raphson Solution (Point load)': v_point,
                   'Newton-Raphson Solution (Continuous load)': v_cont,
                   # Euler-Bernoulli solution for a simply supported beam with a point load at mid-span.
                   'Euler-Bernoulli Solution (Point load)': np.where(x <= L/2, (P * x / (12 * EI)) * ((3/4) * L**2 - x**2), (P * (L - x) / (12 * EI)) * (-x**2 + 2*L*x - (1/4) * L**2)),
                   # Euler-Bernoulli solution for a simply supported beam with a uniformly distributed continuous load.
                   'Euler-Bernoulli Solution (Continuous load)': W*x/(24*EI) * (L**3 - 2*L*x**2 + x**3)})

# Create interactive plot using Plotly Express for visualization.
# This plots the Newton-Raphson solutions against the analytical Euler-Bernoulli solutions.
fig = px.line(df, x='x', y=['Newton-Raphson Solution (Point load)', 'Newton-Raphson Solution (Continuous load)', 'Euler-Bernoulli Solution (Point load)', 'Euler-Bernoulli Solution (Continuous load)'])
fig.show() # Display the interactive plot.

---
Implementation of Point Load and Uniform Continuous Load
---

In [3]:
import numpy as np

def newton_raphson_system(N, init_guess, tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Validate inputs
    valid_init_guesses = ['random', 'uniform', 'zeros']

    if init_guess not in valid_init_guesses:
        raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  2000       # Flexural rigidity (N·m^2)
    P  = -100.0      # Point load at the middle (N)
    W  = -100.0      # Continuous Load (N/m) - This variable is commented out but might be used for continuous load calculations later.
    w  =  0.5        # Relaxation factor - Used to damp the updates in the Newton-Raphson method.

    # Initial guess
    v = np.zeros(N+1)          # Good guess. Achieves Middle Ground.
    v[0] = 0                   # Set boundary condition v(0) = 0
    v[-1] = 0                  # Set boundary condition v(L) = 0

    # Getting the Point Loads
    def get_point_loads():
        """
        Prompts the user to input details for point loads.
        Returns:
            list: A list of tuples, where each tuple contains (position, magnitude) of a point load.
        """
        n = int(input("Enter the number of point loads: "))
        point_loads = []
        for i in range(n):
            pos, mag = map(float, input(f"Enter position and magnitude of point load {i+1} (space-separated): ").split())
            point_loads.append((pos, mag))
        return point_loads

    # Getting the Uniform Continuous Loads
    def get_continuous_loads():
        """
        Prompts the user to input details for continuous loads.
        Returns:
            list: A list of tuples, where each tuple contains (start_position, end_position, intensity) of a continuous load.
        """
        m = int(input("Enter the number of continuous loads: "))
        continuous_loads = []
        for i in range(m):
            start, end, intensity = map(float, input(f"Enter start position, end position, and intensity of continuous load {i+1} (space-separated): ").split())
            continuous_loads.append((start, end, intensity))
        return continuous_loads

    # Generating the Moment Equation
    def moment_equation(L, point_loads, continuous_loads, N):
        """
        Calculates the bending moment along the beam due to applied loads.

        Parameters:
            L (float): Length of the beam.
            point_loads (list): List of (position, magnitude) for point loads.
            continuous_loads (list): List of (start, end, intensity) for continuous loads.
            N (int): Number of grid spaces.

        Returns:
            numpy.ndarray: Array of bending moment values at each discretized point.
        """
        x_vals = np.linspace(0, L, N+1)
        moment_values = np.zeros(N+1)

        for i, x_current in enumerate(x_vals):
            M = 0
            # Calculate moment due to point loads
            for pos, mag in point_loads:
                if not (0 <= pos <= L):
                    raise ValueError(f"Point load at {pos} exceeds beam length {L}.")
                if x_current <= pos:
                    M += mag * (L - pos)/L * x_current
                else:
                    M += mag * pos * (1 - x_current/L)

            # Calculate moment due to continuous loads
            for start, end, intensity in continuous_loads:
                if not (0 <= start <= end <= L):
                    raise ValueError(f"Continuous load from {start} to {end} exceeds beam length {L}.")

                if x_current <= start:
                    M += intensity * (end - start) * (1 - (end + start) / (2 * L)) * x_current
                elif start < x_current < end:
                    M += intensity * (end - start) * (1 - (end + start) / (2 * L)) * x_current - intensity * (x_current - start) * (x_current - start) / 2
                else:
                    M += intensity * (end - start) * (end + start) / (2 * L) * (L - x_current)

            moment_values[i] = M

        return moment_values

    # Residual Function
    def residual(v, M):
        """
        Compute the residual vector R(v) for the nonlinear system.
        The residual represents the error in satisfying the governing differential equation and boundary conditions.
        """
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Apply boundary condition v(0) = 0
        v_full[-1] = 0          # Apply boundary condition v(L) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition for curvature at x=0 (d2v/dx2 = 0) using a forward finite difference approximation
                R[i] = (2*v_full[i]-5*v_full[i+1]+4*v_full[i+2]-v_full[i+3])/dx**2
            elif i == N:
                # Boundary condition for curvature at x=L (d2v/dx2 = 0) using a backward finite difference approximation
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])/dx**2
            else:
                # Discretized governing equation for beam deflection: d2v/dx2 + (M/EI) * (1 + (dv/dx)^2)^(3/2) = 0
                # Using central finite differences for d2v/dx2 and dv/dx
                dv_dx_squared = ((v_full[i+1]-v_full[i-1])/(2*dx))**2 # (dv/dx)^2
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])/dx**2 + (M[i]/EI) * (1 + dv_dx_squared)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """
        Compute the Jacobian matrix J(v) for the nonlinear system.
        The Jacobian contains the partial derivatives of each residual equation with respect to each unknown v_i.
        """
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Apply boundary condition v(0) = 0
        v_full[-1] = 0          # Apply boundary condition v(L) = 0

        for i in range(0, N+1):
            if i == 0:
                # Derivatives of the boundary condition at x=0 with respect to v_i
                # d/dv_0 (2v_0 - 5v_1 + 4v_2 - v_3)/dx^2 = 0 (since v_0 is fixed)
                # d/dv_1 (2v_0 - 5v_1 + 4v_2 - v_3)/dx^2 = -5/dx^2
                # and so on for other terms.
                J[i, i], J[i, i+1], J[i, i+2], J[i, i+3] =  0, -5/dx**2, 4/dx**2, -1/dx**2
            elif i == N:
                # Derivatives of the boundary condition at x=L with respect to v_i
                J[i, i], J[i, i-1], J[i, i-2], J[i, i-3] =  0, -5/dx**2, 4/dx**2, -1/dx**2
            else:
                # Derivatives of the governing equation for intermediate points
                # d/dv_i-1 (R_i)
                # Note: v_full[i-1] is effectively v_{i-1} in the residual equation.
                # The derivative depends on (v_{i+1} - v_{i-1})^2 term.
                if i == 1: # Special case for J[i, i-1] when i=1 as v_0 is fixed
                    J[i, i-1] = 0
                else:
                    J[i, i-1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i-1]-2*v_full[i+1])

                # d/dv_i (R_i)
                J[i, i]   = -2/dx**2

                # d/dv_i+1 (R_i)
                J[i, i+1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Get user input for point loads and continuous loads
    point_loads = get_point_loads()
    continuous_loads = get_continuous_loads()

    # Calculate the bending moment based on the input loads
    moment_values = moment_equation(L, point_loads, continuous_loads, N)
    print()

    # Newton-Raphson iteration loop
    for iteration in range(max_iter):
        # Calculate the residual vector R(v)
        R = residual(v, moment_values)
        print(f"Residual = {np.linalg.norm(R)}") # Print the norm of the residual as a measure of convergence

        # Check for convergence based on residual norm
        if np.linalg.norm(R) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        # Calculate the Jacobian matrix J(v)
        J = jacobian(v, moment_values)
        try:
            # Attempt to solve the linear system J * delta_v = -R for delta_v
            delta_v = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            # If the Jacobian is singular (e.g., matrix is not invertible), use pseudo-inverse (least squares)
            print(f"Jacobian is singular for iteration {iteration+1}. Using pseudo-inverse.\n")
            delta_v = np.linalg.lstsq(J, -R, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        # Update the solution vector v using the relaxation factor w
        v += w*delta_v

        # Check for convergence based on the change in solution vector
        if np.linalg.norm(delta_v) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        # If the loop completes without breaking, it means max_iter was reached without convergence
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Full solution including boundaries (ensure boundary conditions are explicitly applied to the final solution)
    v_full = np.zeros(N+1)
    v_full[:] = v
    v_full[0] = 0
    v_full[-1] = 0

    # Note: P and W are passed back, but W is commented out in this code.
    # If W is intended for continuous load calculation in Euler-Bernoulli, it needs to be defined from user input.
    # For now, it's just a placeholder as it was in the original code.
    return x, v_full, L, P, W, EI


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 250 # Number of grid spaces for discretization
init_guess = input("Enter initial guess type (random, uniform, zeros): ").strip().lower() # Prompt user for initial guess type
x, v, L, P, W, EI = newton_raphson_system(N, init_guess) # Solve the system using Newton-Raphson
print()

# Create Pandas DataFrame to store the results for plotting
df = pd.DataFrame({'x': x, 'Newton-Raphson Solution': v,
                   # Euler-Bernoulli solution for a point load at mid-span for comparison
                   'Euler-Bernoulli Solution (Point load)': np.where(x <= L/2, (P * x / (12 * EI)) * ((3/4) * L**2 - x**2), (P * (L - x) / (12 * EI)) * (-x**2 + 2*L*x - (1/4) * L**2)),
                   # Euler-Bernoulli solution for a uniformly distributed continuous load
                   'Euler-Bernoulli Solution (Continuous load)': W*x/(24*EI) * (L**3 - 2*L*x**2 + x**3)}) # W is -100.0 if not commented out.

# Create interactive plot using Plotly Express
fig = px.line(df, x='x', y=df.columns[1:],
              labels={'value': 'Deflection', 'variable': 'Solution Type'}, # Customize axis and legend labels
              title='Beam Deflection Solutions') # Set plot title

# Update line colors to valid CSS colors (optional customization)
fig.update_traces(line=dict(width=2)) # Set line width for better visibility
fig.update_layout(legend_title_text='Solution Type') # Customize legend title
fig.show() # Display the interactive plot

Enter initial guess type (random, uniform, zeros): uniform
Enter the number of point loads: 0
Enter the number of continuous loads: 0

Residual = 0.0

Converged in 1 iterations.


